# 模型验证和调参

# 读取数据集&特征编码

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn import metrics
# from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
import pickle
# 获取当前的工作目录
pwd = os.getcwd()
# 将工作目录更改到训练集
os.chdir("处理后训练集")
# ——————————————————————————读取训练数据—————————————————————————————— #
# 航班数据
with open('colums_type.pickle','rb') as f:
    colums_types = pickle.load(f)
train_data = pd.read_table('train_data1.csv',sep=',',encoding='gb2312',dtype=colums_types)
os.chdir(pwd)
# 改回原来工作目录

# 填充缺失值
train_data['到达天气'] = train_data['到达天气'].astype('object')
train_data['到达天气']=train_data['到达天气'].fillna('0')
train_data['出发天气'] = train_data['出发天气'].astype('object')
train_data['出发天气']=train_data['出发天气'].fillna('0')
train_data['前序延误']=train_data['前序延误'].fillna(train_data['前序延误'].mean()) #均值填充
train_data['出发气温'] = train_data['出发气温'].astype('object')
train_data['出发气温']=train_data['出发气温'].fillna('一般')
train_data['到达气温'] = train_data['到达气温'].astype('object')
train_data['到达气温']=train_data['到达气温'].fillna('一般')
train_data['起飞间隔'] = train_data['起飞间隔'].fillna(train_data['起飞间隔'].mean())#均值填充
# train_data = train_data.dropna()
# 分类变量
targets = np.where(train_data['起飞延误时间']>3,1,0)

del(train_data['起飞延误时间'])

# ——————————————————————————读取测试数据—————————————————————————————— #
# 工作目录更改到测试集以读取数据
os.chdir("处理后测试集")
# 航班数据
test_data = pd.read_table('test_data1.csv',sep=',',encoding='gb2312')
# 改回工作目录
os.chdir(pwd)
train_columns = train_data.columns
test_data = test_data[train_columns]
# 填充缺失值
test_data['到达天气'] = test_data['到达天气'].astype('object')
test_data['到达天气']=test_data['到达天气'].fillna('0')
test_data['出发天气'] = test_data['出发天气'].astype('object')
test_data['出发天气']=test_data['出发天气'].fillna('0')
test_data['前序延误']=test_data['前序延误'].fillna(test_data['前序延误'].mean()) #均值填充
test_data['出发气温'] = test_data['出发气温'].astype('object')
test_data['出发气温']=test_data['出发气温'].fillna('一般')
test_data['到达气温'] = test_data['到达气温'].astype('object')
test_data['到达气温']=test_data['到达气温'].fillna('一般')
test_data['起飞间隔'] = test_data['起飞间隔'].fillna(train_data['起飞间隔'].mean()) #均值填充

# 编码
# 对字符型数据编码
columns = ['出发机场', '到达机场', '出发天气', '到达天气','航班编号','出发气温','到达气温','航空公司']

le = preprocessing.LabelEncoder()
for col in columns :
    train_data[col] = le.fit_transform(train_data[col])
#     未见过的字符加进去
    clas = list(le.classes_)
    clas = clas + list(test_data[col])
    le.classes_ = np.array(np.unique(clas))
    
    test_data[col] = le.transform(test_data[col])

# 交叉验证

In [5]:
from sklearn import preprocessing
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
#  划分样本集
train_x,test_x,train_y,test_y = train_test_split(train_data,targets,test_size=0.25,random_state=66)
# 设置参数
gbdt = GradientBoostingClassifier(loss='exponential',n_estimators=3,max_depth=5)
# 训练
gbdt.fit(train_x, train_y)
# 预测
pre_y = gbdt.predict_proba(test_x)[:,1]
# 计算auc
fpr, tpr, thresholds = metrics.roc_curve(test_y, pre_y)
auc=metrics.auc(fpr, tpr)
print("AUC得分为：")
print(auc)
# 画出特征重要性图
features = list(train_data.columns)
feature_important = gbdt.feature_importances_
plt.figure(figsize=(12,8))
plt.bar(np.arange(len(features)),feature_important)
plt.xticks(np.arange(len(features)),features,fontsize=12,rotation=10)
plt.show()